In [ ]:
import time
import threading
import queue
import datetime
import schedule
import pymysql
import os

class ScanWorker(threading.Thread):
    def __init__(self, queue, nowdate, num, dbBase, lock):
        threading.Thread.__init__(self)
        self.queue = queue
        self.nowdate = nowdate
        self.num = num
        self.dbBase = dbBase
        self.lock = lock

    def run(self):
        while self.queue.qsize() > 0:
            msgDevInfo = self.queue.get()

            # 取得 lock
            lock.acquire()
            # print("Lock acquired by Worker %d" % self.num)

            # 不能讓多個執行緒同時進的工作
            # print("Worker %d: %s" % (self.num, msg))
            # time.sleep(1)
            # 抓判斷電腦主機狀態與更新-不能讓多個執行緒同時進的工作
            updatePCstatus(msgDevInfo, self.nowdate, self.dbBase)

            # 釋放 lock
            # print("Lock released by Worker %d" % self.num)
            self.lock.release()


# ip lists 最後改撈資料庫

# 建立MySQL連線
def DBConnect():
    dbBase = pymysql.connect(host='140.134.213.105', port=3306, user='root', passwd='Gtml@3777', db='ResourceApp', charset='utf8')
    cursor = dbBase.cursor()
    return dbBase, cursor
            

# 電腦設備IP
def listPC(cursor):
    sqlIp = "SELECT IPAddress1, IsOnline FROM ResourceApp.DeviceConf"
    cursor.execute(sqlIp)
    resultIp = cursor.fetchall()
    return resultIp


def deviceConfSQL():
    sql = " UPDATE ResourceApp.DeviceConf SET UpdateTime = %s, IsOnline = %s WHERE IPAddress1 = %s"
    return sql


# 建立 lock
lock = threading.Lock()

def updatePCstatus(host, nowdate, dbBase):
    print("host:%s, Online:%s" % (host[0], ord(host[1])))
    status = os.system("ping -c 1 " + host[0])
    if status == 0 and ord(host[1]) == 0:
        # update 1
        print("Put isOnline True IP:", host[0])
        val = (nowdate, True, host[0])
        cursor = dbBase.cursor()
        cursor.execute(deviceConfSQL(), val)
        dbBase.commit()
    elif status !=0 and ord(host[1]) == 1:
        # update 0
        print("Put isOnline False IP:", host[0])
        val = (nowdate, False, host[0])
        cursor = dbBase.cursor()
        cursor.execute(deviceConfSQL(), val)
        dbBase.commit()

# 建立Scan電腦是否有開機 job運行
def job():
    nowDate = datetime.datetime.now()
    dbBase, cursor = DBConnect()
    
    hosts = []
    
    # 將IP放到set
    resultIp = listPC(cursor)
    for row in resultIp:
        hosts.append(row)
    # 將IP放到駐列
    ip_queue = queue.Queue()
    for info in hosts:
        ip_queue.put(info)

    # 建立 40 個子執行緒
    threads = []
    for i in range(len(hosts)):
        threads.append(ScanWorker(ip_queue, nowDate, i, dbBase, lock))
        threads[i].start()

    # 等待所有子執行緒結束
    for i in range(len(hosts)):
        threads[i].join()

    # 關閉連線
    cursor.close()
    dbBase.close()
    print("Done. :", datetime.datetime.now())

schedule.every(3).minutes.do(job)

while True:
    schedule.run_pending()
    # time.sleep(1)

host:140.134.211.1, Online:0
host:140.134.211.2, Online:0
host:140.134.211.3, Online:0
host:140.134.211.4, Online:0
host:140.134.211.5, Online:0
host:140.134.211.6, Online:0
host:140.134.211.7, Online:0
host:140.134.211.8, Online:0
host:140.134.211.9, Online:0
host:140.134.211.10, Online:0
host:140.134.211.11, Online:0
host:140.134.211.12, Online:0
host:140.134.211.13, Online:0
host:140.134.211.14, Online:0
host:140.134.211.15, Online:0
host:140.134.211.16, Online:0
host:140.134.211.17, Online:0
host:140.134.211.18, Online:0
host:140.134.211.19, Online:0
host:140.134.211.20, Online:0
host:140.134.211.21, Online:0
host:140.134.211.22, Online:0
host:140.134.211.23, Online:0
host:140.134.211.24, Online:0
host:140.134.211.25, Online:0
host:140.134.211.26, Online:0
host:140.134.211.27, Online:0
host:140.134.211.28, Online:0
host:140.134.211.29, Online:0
host:140.134.211.30, Online:0
